# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
! pip install -U langchain

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [11]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [12]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""
biology_template = """你是一位很棒的生物学家。你擅长回答生物学问题。
之所以如此出色，是因为你能够将复杂的生物学概念分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

computer_science_template = """你是一位很棒的计算机科学家。你擅长回答计算机科学问题。
之所以如此出色，是因为你能够将复杂的计算问题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

chinese_literature_template = """你是一位很棒的汉语言文学家。你擅长回答关于汉语言文学的问题。
之所以如此出色，是因为你能够深入理解文学作品的各个方面，
先回答这些方面的问题，然后再将它们整合起来对作品进行全面的分析。

这是一个问题：
{input}"""

In [13]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
        {
        "name": "生物",
        "description": "适用于回答生物学问题",
        "prompt_template": biology_template
    },
    {
        "name": "计算机科学",
        "description": "适用于回答计算机科学问题",
        "prompt_template": computer_science_template
    },
    {
        "name": "汉语言文学",
        "description": "适用于回答关于汉语言文学的问题",
        "prompt_template":chinese_literature_template
    },
]

In [16]:
import openai
llm = OpenAI(base_url="https://api.xiaoai.plus/v1",api_key="sk-tdqfro61NniG2LZd5394E5F28c51419d8b4dE7Db5a6105C4",model_name="gpt-3.5-turbo-instruct")

In [17]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [18]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [19]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [20]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [21]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物学问题', '计算机科学: 适用于回答计算机科学问题', '汉语言文学: 适用于回答关于汉语言文学的问题']


In [22]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物学问题
计算机科学: 适用于回答计算机科学问题
汉语言文学: 适用于回答关于汉语言文学的问题


In [23]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [24]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物学问题
计算机科学: 适用于回答计算机科学问题
汉语言文学: 适用于回答关于汉语言文学的问题

<< INPUT >>
{i

In [25]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [19]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': '黑体辐射是什么？'}
> Finished chain.
{'input': '黑体辐射是什么？', 'text': '\n\n黑体辐射是指处于热平衡状态的一个理想物体所发出的电磁辐射。根据普朗克定律，黑体辐射的能量密度与频率呈现连续的分布关系，而根据维恩位移定律，黑体辐射的峰值频率与温度成反比。黑体辐射在物理学研究中具有重要的意义，例如在研究热力学系统和量子力学中都有应用。'}


In [20]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '第一个质数是多少，'}
> Finished chain.
{'input': '第一个质数是多少，', 'text': '只能被1和自己整除\n\n首先，我们需要回答什么是质数。质数是指只能被1和自己整除的正整数。\n因此，我们需要从1开始，逐个判断数字是否为质数。\n首先，1不是质数，因为它可以被任何数字整除。\n然后，我们来看数字2，2只能被1和2整除，因此它是质数。\n接下来是数字3，同样只能被1和3整除，所以它也是质数。\n继续这个过程，我们可以得到一系列质数：2、3、5、7、11、13、17、19……\n因此，第一个质数是2。'}


In [21]:
router_chain.verbose = True

In [26]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...
物理: {'input': '什么是黑洞？'}
> Finished chain.
{'input': '什么是黑洞？', 'text': '\n\n黑洞是一种极其密集的天体，它的引力非常强大，甚至连光都无法逃脱。它的密度非常高，可以使空间弯曲，形成一个无法穿越的区域，被称为事件视界。黑洞通常由大型恒星坍塌而成，它们吸收了大量的物质，因此可以被称为宇宙中最强大的“吸星怪”。目前，科学家们还在探索黑洞的性质和起源，这仍然是一个令人充满惊奇的物理现象。 '}




> Entering new MultiPromptChain chain...
生物: {'input': '如何进行能量转换的？'}
> Finished chain.
{'input': '如何进行能量转换的？', 'text': '\n\n能量转换是生物学中非常重要的过程，它指的是将一种形式的能量转换为另一种形式的能量。在生物学中，能量转换通常指的是将光能转换为化学能。生物体通过光合作用来进行能量转换，将太阳光能转换为化学能，用于维持生命活动。\n\n具体来说，光合作用是指植物和某些微生物利用光能将二氧化碳和水转化为有机物和氧气的过程。这个过程中，光能被光合色素吸收，然后通过一系列反应被转换为化学能，最终被储存在分子中，如葡萄糖。这样，植物和微生物就能够利用这些储存的化学能来进行生长和维持生'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains

In [29]:
biology_question = "细胞是如何进行能量转换的？"
chinese_literature_question = "《红楼梦》中的林黛玉代表了什么意象？"
computer_science_question = "什么是面向对象编程？"


In [30]:
print(chain.invoke(biology_question))



> Entering new MultiPromptChain chain...
生物: {'input': '细胞是如何进行能量转换的？'}
> Finished chain.
{'input': '细胞是如何进行能量转换的？', 'text': '\n\n细胞进行能量转换的过程主要是通过细胞呼吸来完成的。细胞呼吸是一种将有机物氧化成二氧化碳和水的生化过程，同时释放出能量的过程。\n\n首先，细胞通过摄取有机物质，如葡萄糖等，来获取能量源。然后，这些有机物质被分解成较小的分子，如乙酸和丙酮酸。这个过程叫做糖酵解，在细胞质中进行。\n\n接着，乙酸和丙酮酸进入线粒体，在线粒体中被进一步氧化成二氧化碳和水。这个过程叫做细胞色素氧化酶系统，在细胞色素氧化酶系统中，产生'}


In [31]:
print(chain.invoke(chinese_literature_question))



> Entering new MultiPromptChain chain...
汉语言文学: {'input': '《红楼梦》中的林黛玉'}
> Finished chain.
{'input': '《红楼梦》中的林黛玉', 'text': '和薛宝钗之间的关系如何？\n\n林黛玉和薛宝钗是《红楼梦》中两位主要的女性角色，她们之间的关系可以说是整部作品的核心之一。林黛玉是贾宝玉的表妹，也是贾府的贵族女儿，性格敏感、多愁善感，有着文艺气质和独特的美感。薛宝钗则是贾宝玉的表妹，贾府的外戚，性格坚强、果断，有着聪慧的头脑和坚强的意志。\n\n从性格上来说，林黛玉和薛宝钗是截然不同的。林黛玉是一个极其敏感的人，她对自己的感情和身'}


In [32]:
print(chain.invoke(computer_science_question))



> Entering new MultiPromptChain chain...
计算机科学: {'input': '什么是面向对象编程？'}
> Finished chain.
{'input': '什么是面向对象编程？', 'text': '\n\n面向对象编程（Object-Oriented Programming，简称OOP）是一种程序设计范式，它以对象作为程序的基本单位，通过将数据和相关操作封装在一起，实现对现实世界的模拟。它允许程序员将复杂的问题分解成多个对象，每个对象具有特定的属性和行为，并通过对象间的交互来实现程序的功能。面向对象编程的核心思想是“万物皆对象”，它强调对象之间的关系和交互，以及对象的继承和多态性。通过对对象进行抽象、封装、继承和多态等操作，面向对象编程可以提高程序的可维护性、可扩展性和重用性，使程序更加稳定、灵活和高效。常见的面向对象编程语言有Java、C++、Python等。'}
